In [1]:
import os
import pandas as pd

if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    for _ in range(2):
        os.chdir(os.path.dirname(os.getcwdb()))


# Consideraciones

- En este "notebook" se trabajan los "dataframes" que relacionan cada imagen con sus características con tal de obtener un único "dataframe" que contenga los datos necesarios para que, una vez identificada la imagen introducida para la predicción, el otro modelo ("XGB"), basado en el "dataframe" original de precios, sea capaz de predecir el precio

- Las imágenes y sus datos se obtienen aquí: https://www.kaggle.com/datasets/harshitlakhani/natural-diamonds-prices-images


# Unión

- Como es más cómodo trabajar con un único "dataframe", se fusionan todos los de los diamantes por características 


In [2]:
df_cushion = pd.read_csv(r'data\raw\images_data\data_cushion.csv')
df_emerald = pd.read_csv(r'data\raw\images_data\data_emerald.csv')
df_heart = pd.read_csv(r'data\raw\images_data\data_heart.csv')
df_marquise = pd.read_csv(r'data\raw\images_data\data_marquise.csv')
df_oval = pd.read_csv(r'data\raw\images_data\data_oval.csv')
df_pear = pd.read_csv(r'data\raw\images_data\data_pear.csv')
df_princess = pd.read_csv(r'data\raw\images_data\data_princess.csv')
df_round = pd.read_csv(r'data\raw\images_data\data_round.csv')

df_diamonds = pd.read_csv(r'data\processed\competition\train_processed.csv')


In [3]:
df_cushion.head()


,Id,Shape,Weight,Clarity,Colour,Cut,Polish,Symmetry,Fluorescence,Messurements,Price
0,1638147,CUSHION,0.55,SI2,E,EX,EX,VG,N,5.05-4.35×2.94,"1,378.65"
1,1630155,CUSHION,0.50,VVS1,FANCY,EX,EX,VG,F,4.60-4.31×2.92,"1,379.74"
2,1612606,CUSHION,0.51,VS2,H,EX,EX,VG,N,4.71-4.35×2.94,"1,380.19"
3,1638140,CUSHION,0.50,VS2,H,EX,EX,VG,N,4.91-4.26×2.88,"1,380.61"
4,1536093,CUSHION,0.53,SI1,D,EX,VG,VG,N,4.70-4.46×3.01,"1,383.13"


In [4]:
dataframes_characteristics_list = [df_cushion, 
                                   df_emerald,
                                   df_heart,
                                   df_marquise,
                                   df_oval,
                                   df_pear,
                                   df_princess,
                                   df_round
                                   ]

df_characteristics = pd.concat(dataframes_characteristics_list)

df_characteristics.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6339 entries, 0 to 2055
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            6339 non-null   object 
 1   Shape         6339 non-null   object 
 2   Weight        6339 non-null   float64
 3   Clarity       6319 non-null   object 
 4   Colour        6339 non-null   object 
 5   Cut           6337 non-null   object 
 6   Polish        6338 non-null   object 
 7   Symmetry      6332 non-null   object 
 8   Fluorescence  6337 non-null   object 
 9   Messurements  6339 non-null   object 
 10  Price         6339 non-null   object 
dtypes: float64(1), object(10)
memory usage: 594.3+ KB


# Adaptación

- Se adapta el "dataframe" resultante para que sea idéntico al que se pasará al modelo "xgboost" para su entreno

In [5]:
df_diamonds.head()


,id,weight (carat),cut quality,color quality,clarity quality,depth (percentage),table (percentage),lenght (millimeters),width (millimeters),depth (millimeters),price
0,0,0.30,3.0,6.0,1.0,62.4,58.0,4.31,4.28,2.68,6.353
1,1,1.01,4.0,5.0,5.0,62.7,56.0,6.42,6.46,4.04,9.183
2,2,0.72,4.0,4.0,3.0,61.8,59.0,5.71,5.74,3.54,7.983
3,3,1.08,2.0,3.0,1.0,63.2,57.0,6.54,6.50,4.12,8.371
4,4,0.36,3.0,3.0,4.0,62.3,59.0,4.50,4.55,2.82,6.588


## Índice

In [6]:
df_characteristics = df_characteristics.set_index('Id')

df_characteristics.head()


,Shape,Weight,Clarity,Colour,Cut,Polish,Symmetry,Fluorescence,Messurements,Price
Id,,,,,,,,,,
1638147,CUSHION,0.55,SI2,E,EX,EX,VG,N,5.05-4.35×2.94,"1,378.65"
1630155,CUSHION,0.50,VVS1,FANCY,EX,EX,VG,F,4.60-4.31×2.92,"1,379.74"
1612606,CUSHION,0.51,VS2,H,EX,EX,VG,N,4.71-4.35×2.94,"1,380.19"
1638140,CUSHION,0.50,VS2,H,EX,EX,VG,N,4.91-4.26×2.88,"1,380.61"
1536093,CUSHION,0.53,SI1,D,EX,VG,VG,N,4.70-4.46×3.01,"1,383.13"


## Borrado de columnas

- Ya existe una columna de precio, pero se borrará para usar el otro modelo con tal de predecirlo, a partir de la imagen que se introduzca, con mayor exactitud, ya que el otro modelo se entrena con datos que este primero no ve y tiene ocho veces los diamantes que constan en este. Tratar de predecir el precio directamente de la imagen podría dar lugar a un mayor margen de error

- Se ha visto en "Unión" que hay nulos, además de columnas que no sirven. No queda más remedio que descartarlos

In [7]:
df_characteristics = df_characteristics.drop(columns=['Polish', 'Symmetry', 'Fluorescence', 'Price', 'Shape'])

df_characteristics.head()


,Weight,Clarity,Colour,Cut,Messurements
Id,,,,,
1638147,0.55,SI2,E,EX,5.05-4.35×2.94
1630155,0.50,VVS1,FANCY,EX,4.60-4.31×2.92
1612606,0.51,VS2,H,EX,4.71-4.35×2.94
1638140,0.50,VS2,H,EX,4.91-4.26×2.88
1536093,0.53,SI1,D,EX,4.70-4.46×3.01


In [8]:
df_characteristics = df_characteristics.dropna()

df_characteristics.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6317 entries, 1638147 to 1630512
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Weight        6317 non-null   float64
 1   Clarity       6317 non-null   object 
 2   Colour        6317 non-null   object 
 3   Cut           6317 non-null   object 
 4   Messurements  6317 non-null   object 
dtypes: float64(1), object(4)
memory usage: 296.1+ KB


## Columna "Cut"

- Se pasa la columna a números

- No hay el equivalente a "Premium" del "dataframe" de predicción de precios, así que se ignora ese número


In [9]:
df_characteristics['Cut'].unique()


array(['EX', 'VG', 'GD', 'F'], dtype=object)

In [10]:
df_characteristics.loc[:, 'Cut'] = df_characteristics['Cut'].apply(lambda x: 0 if x == 'F' 
                                                                    else 1 if x == 'GD'
                                                                    else 2 if x == 'VG'
                                                                    else 4
                                                                    )
df_characteristics.loc[:, 'Cut'] = df_characteristics['Cut'].astype(float)

df_characteristics['Cut'].unique()

C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\1055745526.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_characteristics.loc[:, 'Cut'] = df_characteristics['Cut'].apply(lambda x: 0 if x == 'F'
C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\1055745526.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_characteristics.loc[:, 'Cut'] = df_characteristics['Cut'].astype(float)


array([4., 2., 1., 0.])

## Columna "Colour"

- Hay muchas más posibilidades de color que en el "dataframe" de precios, así que se descartan 

In [11]:
df_characteristics['Colour'].unique()


array(['E', 'FANCY', 'H', 'D', 'I', 'M', 'K', 'W-X', 'Y-Z', 'N', 'J', 'F',
       'L', 'G', 'O-P', 'S-T', 'U-V', 'W', 'Q-R', 'O'], dtype=object)

In [12]:
df_characteristics = df_characteristics.drop(df_characteristics[df_characteristics['Colour'] == 'FANCY'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'M'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'K'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'W-X'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'Y-Z'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'N'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'L'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'O-P'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'S-T'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'U-V'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'W'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'Q-R'].index
                                                | df_characteristics[df_characteristics['Colour'] == 'O'].index                                                                                  
                                                )

df_characteristics['Colour'].unique()


C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\3476073297.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  df_characteristics = df_characteristics.drop(df_characteristics[df_characteristics['Colour'] == 'FANCY'].index
C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\3476073297.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  df_characteristics = df_characteristics.drop(df_characteristics[df_characteristics['Colour'] == 'FANCY'].index
C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\3476073297.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  df_characteristics = df_characteristics.drop(d

array(['E', 'H', 'D', 'I', 'J', 'F', 'G'], dtype=object)

In [13]:
df_characteristics.loc[:, 'Colour'] = df_characteristics['Colour'].apply(lambda x: 0 if x == 'J'
                                                else 1 if x == 'I' 
                                                else 2 if x == 'H' 
                                                else 3 if x == 'G'
                                                else 4 if x == 'F'
                                                else 5 if x == 'E'
                                                else 6)
df_characteristics.loc[:, 'Colour'] = df_characteristics['Colour'].astype(float)

df_characteristics['Colour'].unique()


C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\1919486356.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_characteristics.loc[:, 'Colour'] = df_characteristics['Colour'].apply(lambda x: 0 if x == 'J'
C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\1919486356.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_characteristics.loc[:, 'Colour'] = df_characteristics['Colour'].astype(float)


array([5., 2., 6., 1., 0., 4., 3.])

## Columna "Clarity"

- No existe la claridad "FL" en el "dataframe" de precios, así que se descarta

In [14]:
df_characteristics['Clarity'].unique()


array(['SI2', 'VS2', 'SI1', 'VVS1', 'VVS2', 'VS1', 'IF', 'I1', 'FL'],
      dtype=object)

In [15]:
df_characteristics = df_characteristics.drop(df_characteristics[df_characteristics['Clarity'] == 'FL'].index)

df_characteristics['Clarity'].unique()


array(['SI2', 'VS2', 'SI1', 'VVS1', 'VVS2', 'VS1', 'IF', 'I1'],
      dtype=object)

In [16]:
df_characteristics.loc[:, 'Clarity'] = df_characteristics['Clarity'].apply(lambda x: 0 if x == 'I1'
                                                                            else 1 if x == 'SI2'
                                                                            else 2 if x == 'SI1'
                                                                            else 3 if x == 'VS2'
                                                                            else 4 if x == 'VS1'
                                                                            else 5 if x == 'VVS2'
                                                                            else 6 if x == 'VVS1'
                                                                            else 7
                                                                            )
df_characteristics.loc[:, 'Clarity'] = df_characteristics['Clarity'].astype(float)

df_characteristics['Clarity'].unique()


C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\3580306968.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_characteristics.loc[:, 'Clarity'] = df_characteristics['Clarity'].apply(lambda x: 0 if x == 'I1'
C:\Users\Roger\AppData\Local\Temp\ipykernel_8388\3580306968.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_characteristics.loc[:, 'Clarity'] = df_characteristics['Clarity'].astype(float)


array([1., 3., 2., 6., 5., 4., 7., 0.])

## Columna "Messurements"

- Hay que dividirla en las 3 correspondientes del "dataframe" de precios

- Con los datos resultantes, hay que calcular la columna de "depth (percentage)" mediante la operación aprendida en el EDA

In [17]:
df_characteristics['Messurements'].unique()


array(['5.05-4.35×2.94', '4.71-4.35×2.94', '4.91-4.26×2.88', ...,
       '5.00-5.04×3.18', '4.93-4.96×3.19', '5.09-5.13×3.12'], dtype=object)

In [18]:
df_characteristics['lenght (millimeters)'] =  df_characteristics['Messurements'].str.extract('^([0-9]?.?[0-9]+)')

df_characteristics['lenght (millimeters)'].unique()


array(['5.05', '4.71', '4.91', '4.70', '4.76', '5.16', '4.48', '4.55',
       '4.89', '4.69', '4.78', '4.64', '4.63', '4.53', '5.13', '4.67',
       '4.62', '5.19', '5.01', '4.57', '4.96', '5.28', '5.34', '4.49',
       '4.77', '4.95', '4.66', '3.95', '4.25', '4.97', '4.98', '4.59',
       '4.44', '4.93', '4.60', '4.92', '4.84', '5.18', '4.73', '4.94',
       '4.56', '4.74', '4.99', '4.86', '4.61', '5.03', '5.06', '5.08',
       '4.52', '4.43', '4.82', '4.58', '5.55', '4.87', '5.20', '4.72',
       '4.47', '5.02', '4.88', '4.75', '5.22', '5.23', '5.17', '5.21',
       '4.81', '4.68', '5.04', '5.53', '5.24', '4.45', '4.39', '5.44',
       '4.51', '5.35', '5.00', '5.52', '4.90', '5.15', '4.50', '4.65',
       '5.10', '4.54', '5.11', '5.29', '4.83', '4.32', '4.22', '5.46',
       '5.62', '5.32', '5.36', '5.30', '5.54', '5.37', '5.73', '5.64',
       '5.51', '5.67', '5.60', '5.80', '5.83', '5.43', '5.45', '5.88',
       '6.06', '5.78', '5.26', '5.09', '5.56', '5.42', '5.58', '5.12',
      

In [19]:
df_characteristics['width (millimeters)'] =  df_characteristics['Messurements'].str.extract('[\sX\-]+([0-9]?.?[0-9]+)[\sxX×]+')

df_characteristics['width (millimeters)'].unique()


array(['4.35', '4.26', '4.46', '4.42', '4.21', '4.17', '4.38', '4.50',
       '4.24', '4.52', '4.48', '4.33', '4.43', '4.47', '4.12', '4.28',
       '4.40', '4.45', '4.29', '4.20', '4.81', '4.27', '4.32', '4.34',
       '4.00', '3.89', '3.98', '3.78', '4.06', '4.53', '4.25', '4.16',
       '4.13', '4.37', '4.49', '4.31', '4.39', '4.19', '4.51', '4.23',
       '4.22', '4.79', '4.44', '4.09', '4.36', '4.86', '4.18', '4.95',
       '4.72', '4.78', '4.83', '4.62', '4.41', '4.14', '4.84', '4.67',
       '4.30', '4.73', '4.58', '4.54', '3.92', '3.84', '3.97', '4.60',
       '4.11', '4.96', '5.01', '4.55', '5.11', '4.90', '4.10', '5.14',
       '5.16', '4.94', '4.82', '5.08', '5.26', '5.06', '4.98', '4.63',
       '4.93', '5.10', '4.71', '5.04', '5.00', '4.68', '5.12', '4.89',
       '4.88', '5.03', '4.87', '5.52', '5.30', '5.15', '4.80', '4.66',
       '4.85', '5.29', '4.74', '5.02', '5.05', '4.99', '4.91', '5.19',
       '5.47', '4.77', '5.23', '5.13', '4.59', '5.18', '5.09', '4.75',
      

In [20]:
df_characteristics['depth (millimeters)'] =  df_characteristics['Messurements'].str.extract('[\sX\-]+[0-9]?.?[0-9]+[\sxX×]+([0-9]?.?[0-9]+)')

df_characteristics['depth (millimeters)'].unique()


array(['2.94', '2.88', '3.01', '2.99', '2.85', '2.90', '3.04', '3.02',
       '2.84', '2.83', '3.03', '2.92', '3.00', '2.78', '2.79', '2.87',
       '3.26', '2.86', '2.68', '2.64', '2.53', '2.74', '3.06', '2.98',
       '2.80', '2.97', '2.81', '2.95', '2.89', '3.05', '2.93', '3.07',
       '2.96', '2.82', '3.25', '2.73', '3.30', '3.33', '3.29', '3.28',
       '3.13', '3.34', '3.20', '3.12', '3.22', '2.91', '3.08', '3.27',
       '2.56', '3.11', '3.21', '3.37', '3.44', '3.09', '3.42', '3.49',
       '3.47', '3.41', '3.45', '3.56', '3.43', '3.38', '3.17', '3.31',
       '3.35', '3.18', '3.59', '3.50', '3.52', '3.16', '3.40', '3.15',
       '3.55', '3.51', '3.32', '3.14', '3.46', '3.48', '3.54', '3.71',
       '3.24', '3.36', '3.66', '3.53', '3.69', '3.68', '3.39', '3.58',
       '3.19', '3.83', '3.67', '3.57', '3.23', '3.72', '3.70', '3.63',
       '3.76', '3.64', '4.16', '4.39', '3.62', '3.75', '3.74', '3.65',
       '3.61', '3.77', '3.79', '4.02', '3.97', '4.52', '3.60', '3.92',
      

In [21]:
df_characteristics[df_characteristics['depth (millimeters)'].isna()]

,Weight,Clarity,Colour,Cut,Messurements,lenght (millimeters),width (millimeters),depth (millimeters)
Id,,,,,,,,


In [22]:
df_characteristics = df_characteristics.drop(columns='Messurements')

df_characteristics.head()


,Weight,Clarity,Colour,Cut,lenght (millimeters),width (millimeters),depth (millimeters)
Id,,,,,,,
1638147,0.55,1.0,5.0,4.0,5.05,4.35,2.94
1612606,0.51,3.0,2.0,4.0,4.71,4.35,2.94
1638140,0.50,3.0,2.0,4.0,4.91,4.26,2.88
1536093,0.53,2.0,6.0,4.0,4.70,4.46,3.01
1643527,0.52,6.0,1.0,4.0,4.76,4.42,2.99


In [23]:
df_characteristics = df_characteristics.apply(pd.to_numeric)

df_characteristics['depth (percentage)'] = (df_characteristics['depth (millimeters)'] / ((df_characteristics['lenght (millimeters)'] + df_characteristics['width (millimeters)']) / 2)) * 100

df_characteristics['depth (percentage)'].unique()


array([62.55319149, 64.90066225, 62.81352236, ..., 59.90338164,
       60.50583658, 63.34661355])

## Nuevos nombres

In [24]:
df_characteristics = df_characteristics.rename(columns={'Weight': 'weight (carat)',
                                                        'Cut': 'cut quality',
                                                        'Colour': 'color quality',
                                                        'Clarity': 'clarity quality',
                                                        }
                                                )

df_characteristics.head()


,weight (carat),clarity quality,color quality,cut quality,lenght (millimeters),width (millimeters),depth (millimeters),depth (percentage)
Id,,,,,,,,
1638147,0.55,1.0,5.0,4.0,5.05,4.35,2.94,62.553191
1612606,0.51,3.0,2.0,4.0,4.71,4.35,2.94,64.900662
1638140,0.50,3.0,2.0,4.0,4.91,4.26,2.88,62.813522
1536093,0.53,2.0,6.0,4.0,4.70,4.46,3.01,65.720524
1643527,0.52,6.0,1.0,4.0,4.76,4.42,2.99,65.141612


## Cambio de orden

In [25]:
df_characteristics = df_characteristics[['weight (carat)', 'cut quality', 'color quality', 'clarity quality', 'depth (percentage)', 'lenght (millimeters)', 'width (millimeters)', 'depth (millimeters)']]

df_characteristics.head()


,weight (carat),cut quality,color quality,clarity quality,depth (percentage),lenght (millimeters),width (millimeters),depth (millimeters)
Id,,,,,,,,
1638147,0.55,4.0,5.0,1.0,62.553191,5.05,4.35,2.94
1612606,0.51,4.0,2.0,3.0,64.900662,4.71,4.35,2.94
1638140,0.50,4.0,2.0,3.0,62.813522,4.91,4.26,2.88
1536093,0.53,4.0,6.0,2.0,65.720524,4.70,4.46,3.01
1643527,0.52,4.0,1.0,6.0,65.141612,4.76,4.42,2.99


## Eliminación de duplicados

In [26]:
df_characteristics = df_characteristics.drop_duplicates()


# Exportación

- Se guarda el nuevo "dataframe" como "csv" listo para el modelaje

- Como no hay columna "table (percentage)", no queda más remedio que reentrenar el modelo "XGBoost" sin esa columna. En ese sentido, se usa el "dataset" original en lugar del de competición para un mejor resultado

In [27]:
df_characteristics.to_csv(r'data\processed\images_data_processed.csv', index='Id')
